# Debugging MAL
* When MAL Uploads aren't working you can follow some of the steps in this guide

#### 1: Make sure that model assisted labeling is turned on for your project.
    * You can do that from the web ui or with the following code:
``` project.enable_model_assisted_labeling()```

#### 2: Make sure your import objects or import file matches the following structure:
    * https://docs.labelbox.com/en/model-assisted-labeling/create-import-file

#### 3: Make sure your import objects or import file matches the following:
    * Make sure validate is set to true (it is by default) on your upload (Make sure you are using sdk version > 2.5.1)

    

#### Reading validation Errors:
* We get an mal validation error:
* ```MALValidationError: Invalid NDJson on line 0('Invalid NDJson on line 0', None)```
* This means that line 0 (the first example), was invalid.
---
* We can see why it was invalid up above the MALValidationError. 
* The origianal error was a ValuError because we are attempting to attach an example to a data row that is not in the project
--- 
* Errors will be thrown for incorrect or missing keys
* invalid datarow or schema ids
* invalid data types


In [1]:
!pip install labelbox
!pip install ndjson
!pip install requests

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [2]:
# Note that the following code (_validate_ndjson) is run when you perform bulk uploads. 
# You don't need to use it directly. This section will show you how to read the error
from labelbox.schema.bulk_import_request import _validate_ndjson
from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox import Client, LabelingFrontend
import ndjson
import requests
import uuid
import os

In [3]:
# Set this if running in colab. Otherwise it should work if you have the LABELBOX_API_KEY set.
API_KEY = os.environ["LABELBOX_API_KEY"]
# Only update this if you have an on-prem deployment
ENDPOINT = "https://api.labelbox.com/graphql"

In [4]:
client = Client(api_key = API_KEY, endpoint = ENDPOINT)

In [5]:
ontology_builder = OntologyBuilder(
    tools = [
        Tool(tool = Tool.Type.POLYGON, name = "person"),             
    ]
)

In [6]:
project = client.create_project(name = "debugging_mal_project")
dataset = client.create_dataset(name = "debugging_mal_dataset")
test_img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Kitano_Street_Kobe01s5s4110.jpg/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data = test_img_url)
editor = next(client.get_labeling_frontends(where = LabelingFrontend.name == 'editor'))
project.setup(editor, ontology_builder.asdict())
project.datasets.connect(dataset)
ontology = ontology_builder.from_project(project)

In [7]:
poly_example = {
    "uuid": "1b5762e9-416c-44cf-9a5f-07effb51f863",
    "schemaId": ontology.tools[0].feature_schema_id,
    "dataRow": {
         "id": "cjxav4aa07r1g0dsq70t9eveg" #Invalid (not attached to this project) data_row.uid
     },
     "polygon": [{"x": 2,"y": 99},{"x": 93,"y": 5},{"x": 51,"y": 106},{"x": 176,"y": 142}]
}

In [8]:
list(list(project.datasets())[0].data_rows())[0].uid

'ckmunvlqt4w9u0y625l7cd7zq'

In [9]:
_validate_ndjson([poly_example], project)

MALValidationError: Invalid NDJson on line 0('Invalid NDJson on line 0', None)

#### Reading MAL Errors
* If your upload passes client side validation checks, there is still a chance of server side failures.
* There are two urls for checking the status of your upload
    * bulk_import_request.status_file_url
    * bulk_import_request.error_file_url

In [10]:
invalid_example = poly_example.copy()
invalid_example['polygon'] = [{"x": 2,"y": 99}]

# Old way:
name = str(uuid.uuid4())
bulk_import_request = project.upload_annotations(
    name=name, 
    annotations=[poly_example, invalid_example], 
    validate = False # Turn this off or it will raise the error on the client side
)
bulk_import_request.wait_until_done()


In [11]:
# Let's check the status:
for status in bulk_import_request.statuses:
    print(status)
    
for error in bulk_import_request.errors:
    print(error)

{'uuid': '1b5762e9-416c-44cf-9a5f-07effb51f863', 'dataRow': {'id': 'cjxav4aa07r1g0dsq70t9eveg'}, 'status': 'FAILURE', 'errors': [{'name': 'DataRowNotFound', 'message': 'dataRow.id cjxav4aa07r1g0dsq70t9eveg invalid', 'additionalInfo': None}]}
{'uuid': '1b5762e9-416c-44cf-9a5f-07effb51f863', 'dataRow': {'id': 'cjxav4aa07r1g0dsq70t9eveg'}, 'status': 'FAILURE', 'errors': [{'name': 'DataRowNotFound', 'message': 'dataRow.id cjxav4aa07r1g0dsq70t9eveg invalid', 'additionalInfo': None}, {'name': 'GeometryInvalid', 'message': 'Invalid geometry: Each linear ring must contain at least 4 positions', 'additionalInfo': None}]}


In [ ]:
# We can see that we are using invalid datarows (they don't belong to the project we are uploading to)
# and that the geometry is invalid.